#SAM (2023)

[[arxiv] 🎓 Segment Anything (Kirillov et al., 2023)](https://arxiv.org/abs/2304.02643)

Модель возвращает набор масок, соответствующих входу. Классы объектов не используются.

В качестве входа могут подаваться:

*  набор точек,
*  набор bounding box,
*  маски,
*  текст (поддержка в коде пока не реализована),
*  изображение.


<img src ="https://ml.gan4x4.ru/msu/dep-2.1/L11/sam_overview.png" width="1000">

Обучалась на огромном датасете, частично размеченном в unsupervise режиме.

<img src ="https://ml.gan4x4.ru/msu/dep-2.1/L11/sam_architecture.png" width="1000">

Установим пакет:

In [ ]:
!pip install -q git+https://github.com/facebookresearch/segment-anything.git

Загружаем веса из [репозитория Facebook Research 🐾[git]](https://github.com/facebookresearch/segment-anything#model-checkpoints):

In [ ]:
# ViT-H
!wget -nc https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

Создаем encoder:

In [ ]:
import torch
from segment_anything import sam_model_registry
from warnings import simplefilter

simplefilter("ignore", category=FutureWarning)

# model_type = "vit_h"
sam = sam_model_registry["vit_h"](checkpoint="sam_vit_h_4b8939.pth")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sam.to(device=device)
print("Checkpoit loaded") #suppres printing model structure

Загрузим изображение:

In [ ]:
# Source: http://images.cocodataset.org/val2017/000000448263.jpg
!wget -qN https://ml.gan4x4.ru/msu/dep-2.1/L11/000000448263.jpg

In [ ]:
import numpy as np
from PIL import Image

img = Image.open("000000448263.jpg")
np_im = np.array(img)  # HWC format
img

Создадим эмбеддинг (на CPU выполняется долго) и предскажем все маски.

[[git] 🐾 Automatically generating object masks with SAM (example)](https://github.com/facebookresearch/segment-anything/blob/main/notebooks/automatic_mask_generator_example.ipynb)

In [ ]:
%%time
from segment_anything import SamAutomaticMaskGenerator

mask_generator = SamAutomaticMaskGenerator(sam)
masks = mask_generator.generate(np_im)

На выходе получаем список:

In [ ]:
masks[0]

In [ ]:
masks[0]["segmentation"].shape

In [ ]:
# https://github.com/facebookresearch/segment-anything/blob/main/notebooks/automatic_mask_generator_example.ipynb
import matplotlib.pyplot as plt


def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x["area"]), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones(
        (
            sorted_anns[0]["segmentation"].shape[0],
            sorted_anns[0]["segmentation"].shape[1],
            4,
        )
    )
    img[:, :, 3] = 0
    for ann in sorted_anns:
        m = ann["segmentation"]
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

In [ ]:
plt.figure(figsize=(10, 8))
plt.imshow(img)
show_anns(masks)
plt.axis("off")
plt.show()

Предсказываем по точкам. Сначала создаем эмбеддинг. Он хранится внутри модели.

[[git] 🐾 Object masks from prompts with SAM (example)](https://github.com/facebookresearch/segment-anything/blob/main/notebooks/predictor_example.ipynb)

In [ ]:
%%time
from segment_anything import SamPredictor


predictor = SamPredictor(sam)
predictor.set_image(np_im)  # create embedding

Теперь получаем предсказания, указав точки, которые относятся к объекту и фону:

In [ ]:
masks, scores, logits = predictor.predict(
    point_coords=np.array([[200, 200], [1, 1]]),  # point coords
    point_labels=np.array([1, 0]),  # 1 - object(foreground), 0 - background
    # box
    # mask_input
    multimask_output=True,  # return 1 or 3 masks because of the ambiguous input
)

In [ ]:
print("Masks count", len(masks))
print("Scores", scores)

In [ ]:
print(masks[0].shape)

In [ ]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30 / 255, 144 / 255, 255 / 255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)


plt.imshow(img)
show_mask(masks[2], plt.gca())
plt.axis("off")
plt.show()